In [ ]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec

# Load environment variables from a .env file
load_dotenv()





In [ ]:
pc = Pinecone(api_key= os.getenv("PINECONE_API_KEY"))
try:
    pc.create_index(
        name="rag",
        dimension = 1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )
except:
    pass 

In [ ]:
import json
data = json.load(open("reviews.json"))

data['reviews']

In [ ]:
from mixedbread_ai.client import MixedbreadAI
processed_data=[]

from openai import OpenAI

# Initialize OpenAI client using OpenRouter's base URL and API key from environment variable
mxbai = MixedbreadAI(api_key = os.getenv('MIXEDBREAD_API_KEY'))

for review in data['reviews'] :
    response = mxbai.embeddings(
        input = review['comments'],
        model = "mixedbread-ai/mxbai-embed-large-v1",
        normalized = True,
        encoding_format = 'float',
        truncation_strategy = 'end'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        'values': embedding,
        'id' : review['name'],
        'metadata': {
            'review' : review['comments'],
            'skills': review['skills'],
            'stars': review['stars'] 
        }   
    })

In [ ]:
processed_data[0]

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data,
    namespace = 'ns1'
)